# Surrounding Venues for Each Arcade

__Skip to the Good Stuff__

[Importing Libraries](#Importing-Libraries)

[Loading Arcade DataFrame](#Loading-Arcade-DataFrame)

[Connecting to FourSquare](#Connecting-to-FourSquare)

[Making Surrounding Venues DataFrame](#Making-Surrounding-Venues-DataFrame)

[Dropping the Nulls](#Dropping-the-Nulls)

[Sorting by Arcade and Distance](#Sorting-by-Arcade-and-Distance)

[Ensuring Unique Venues](#Ensuring-Unique-Venues)

[Pulling the Ratings](#Pulling-the-Ratings)

### Importing Libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

! pip install folium==0.5.0
import folium # plotting library

import json

pd.set_option('display.max_rows', None) #To view the full dataframe

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Loading Arcade DataFrame

In [2]:
seattle_arcades = pd.read_csv('Arcade_final_clean.csv')
seattle_arcades = seattle_arcades.drop('Unnamed: 0', axis = 1)
print(len(seattle_arcades))
seattle_arcades

15


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,University District,4af30888f964a520f5e921e3,4518 Arcade,59,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office,Not Rated
1,Lower Queen Anne,4d03d2b6c2e537042963b467,Penny Arcade Warehouse,473,3150 Elliott Ave,47.618242,-122.357470,4bf58dd8d48988d1e1931735,Arcade,Not Rated
2,Highland Park,4dbf6f1e6a23e5a549d6141b,SanTe's Arcade,752,No Address,47.524985,-122.361241,4bf58dd8d48988d1e1931735,Arcade,Not Rated
3,Pioneer Square,4e53fe8522711d4da5c38798,Grand Central Building - Lower Arcade,186,No Address,47.600463,-122.333970,5744ccdfe4b0c0459246b4dc,Shopping Plaza,Not Rated
4,Fremont,4ee59ef76c25be9631860f8d,Add-a-Ball Arcade,399,315 N 36th St #2B,47.652040,-122.354771,4bf58dd8d48988d1e1931735,Arcade,8.8
5,Pioneer Square,4fb7f757e4b09a1e8cab118e,Time masheen arcade,252,No Address,47.601545,-122.330684,4bf58dd8d48988d1e1931735,Arcade,Not Rated
6,Lower Queen Anne,4fdd404be4b082d0767d2155,Seattle Interactive Media Museum,387,No Address,47.621242,-122.350853,4bf58dd8d48988d1e1931735,Arcade,Not Rated
7,Renton Hill,4ff503b1e4b02ca33b4b731f,Rod's Pinball Arcade,829,9818 Beacon Ave S,47.512992,-122.272850,4bf58dd8d48988d1e1931735,Arcade,Not Rated
8,Lower Queen Anne,5064c506e4b06062642548ae,Next 50 Video Arcade,390,Seattle Center,47.621220,-122.350827,4bf58dd8d48988d1e1931735,Arcade,Not Rated
9,Harrison,50fa31bae4b020b5559a65cc,Lipton's Arcade,373,No Address,47.618770,-122.326777,4bf58dd8d48988d11a941735,Other Nightlife,Not Rated


### Connecting to FourSquare

In [3]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [4]:
VERSION = '20180604'
LIMIT = 20

### Making Surrounding Venues DataFrame

In [5]:
#assign variables for the search
search_query = ''
radius = 50
print(search_query + ' .... OK!')

 .... OK!


In [6]:
surrounding_venues = pd.DataFrame(columns = ['Arcade','Venue_ID','Venue_Name','Venue_Distance','Venue_Address','Venue_Latitude','Venue_Longitude','Category_ID','Category_Name'])
surrounding_venues

,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name


In [7]:
for index, arcade in enumerate(seattle_arcades['Venue_ID']):
    try:
        arcade_name = seattle_arcades['Venue_Name'][index]
        latitude = seattle_arcades['Venue_Latitude'][index]
        longitude = seattle_arcades['Venue_Longitude'][index]
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
    except:
        continue
    try:
        res = results['response']['venues']
    except:
        print('No response')
    for dictionary in results['response']['venues']:
        try:
            ident= dictionary['id']
        except:
            ident= 'No ID'
        try:
            name = dictionary['name']
        except:
            name = 'No Name'
        try:
            ven_add = dictionary['location']['address']
        except:
            ven_add = 'No Address'
        try:
            ven_lat = dictionary['location']['lat']
        except:
            ven_lat = 'No Latitude'
        try:
            ven_lng = dictionary['location']['lng']
        except:
            ven_lng = 'No Longitude'
        try:
            ven_dist = dictionary['location']['distance']
        except:
            ven_dist = 'No Distance'
        try:
            for cat in dictionary['categories']:
                try:
                    cat_id= cat['id']
                except:
                    cat_id= 'No ID'
                try:
                    cat_name = cat['name']
                except:
                    cat_name = 'No Name'
        except:
            continue
        try:
            surrounding_venues = surrounding_venues.append({'Venue_Distance':ven_dist,'Arcade':arcade_name, 'Venue_ID':ident, 'Venue_Name':name,'Venue_Address':ven_add,'Venue_Latitude':ven_lat,'Venue_Longitude':ven_lng,'Category_ID':cat_id,'Category_Name':cat_name}, ignore_index=True)
        except:
            surrounding_venues = surrounding_venues.append({'Arcade': 'Null'}, ignore_index=True)

In [8]:
print(len(surrounding_venues))
surrounding_venues.head()

300


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,4518 Arcade,4ef5323b77c8e9126478ff6f,U:DON Fresh Japanese Noodle Station,7,4515 University Way NE,47.661795,-122.313194,55a59bace4b013909087cb2a,Udon Restaurant
1,4518 Arcade,4ae4f7d0f964a520c59f21e3,Urban Outfitters,45,4518 University Way Northeast,47.661430,-122.313030,4bf58dd8d48988d103951735,Clothing Store
2,4518 Arcade,59c14ab61ffed75cd77ec327,Chi Mac,25,4525 University Way NE,47.662044,-122.313256,4bf58dd8d48988d113941735,Korean Restaurant
3,4518 Arcade,5abb2319b040560b822d9633,Taste of Xi'an,21,4523 University Way NE,47.661957,-122.313337,4bf58dd8d48988d145941735,Chinese Restaurant
4,4518 Arcade,5f0e45adc5d2fb4660d1b12e,Don't Yell At Me 不要對我尖叫,21,4511 University Way NE,47.661674,-122.313270,52e81612bcbc57f1066b7a0c,Bubble Tea Shop


In [9]:
surrounding_venues.to_csv('surrounding_raw.csv')

### Dropping the Nulls

In [24]:
surrounding_venues = pd.read_csv('surrounding_raw.csv')
surrounding_venues = surrounding_venues.drop('Unnamed: 0', axis= 1)
print(len(surrounding_venues))
surrounding_venues.head()

300


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,4518 Arcade,4ef5323b77c8e9126478ff6f,U:DON Fresh Japanese Noodle Station,7,4515 University Way NE,47.661795,-122.313194,55a59bace4b013909087cb2a,Udon Restaurant
1,4518 Arcade,4ae4f7d0f964a520c59f21e3,Urban Outfitters,45,4518 University Way Northeast,47.661430,-122.313030,4bf58dd8d48988d103951735,Clothing Store
2,4518 Arcade,59c14ab61ffed75cd77ec327,Chi Mac,25,4525 University Way NE,47.662044,-122.313256,4bf58dd8d48988d113941735,Korean Restaurant
3,4518 Arcade,5abb2319b040560b822d9633,Taste of Xi'an,21,4523 University Way NE,47.661957,-122.313337,4bf58dd8d48988d145941735,Chinese Restaurant
4,4518 Arcade,5f0e45adc5d2fb4660d1b12e,Don't Yell At Me 不要對我尖叫,21,4511 University Way NE,47.661674,-122.313270,52e81612bcbc57f1066b7a0c,Bubble Tea Shop


In [25]:
surrounding_venues = surrounding_venues[surrounding_venues != 'Null']
surrounding_venues = surrounding_venues[surrounding_venues != '[]']
surrounding_venues = surrounding_venues.dropna()
print(len(surrounding_venues))
surrounding_venues = surrounding_venues.reset_index()
surrounding_venues = surrounding_venues.drop(['index'], axis = 1)
surrounding_venues.head(20)

300


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,4518 Arcade,4ef5323b77c8e9126478ff6f,U:DON Fresh Japanese Noodle Station,7,4515 University Way NE,47.661795,-122.313194,55a59bace4b013909087cb2a,Udon Restaurant
1,4518 Arcade,4ae4f7d0f964a520c59f21e3,Urban Outfitters,45,4518 University Way Northeast,47.661430,-122.313030,4bf58dd8d48988d103951735,Clothing Store
2,4518 Arcade,59c14ab61ffed75cd77ec327,Chi Mac,25,4525 University Way NE,47.662044,-122.313256,4bf58dd8d48988d113941735,Korean Restaurant
3,4518 Arcade,5abb2319b040560b822d9633,Taste of Xi'an,21,4523 University Way NE,47.661957,-122.313337,4bf58dd8d48988d145941735,Chinese Restaurant
4,4518 Arcade,5f0e45adc5d2fb4660d1b12e,Don't Yell At Me 不要對我尖叫,21,4511 University Way NE,47.661674,-122.313270,52e81612bcbc57f1066b7a0c,Bubble Tea Shop
5,4518 Arcade,5e23b504daee0b0008f60baa,Sugar+Spoon Cookie Dough,5,No Address,47.661790,-122.313146,4bf58dd8d48988d1c9941735,Ice Cream Shop
6,4518 Arcade,4b58fe5ff964a5205e7728e3,Deep Roots Tattoo & Body Piercing,5,4517 University Way NE,47.661806,-122.313184,4bf58dd8d48988d1de931735,Tattoo Parlor
7,4518 Arcade,4c37a5ae93db0f47db2f2092,Metro Bus Stop #9584,21,University Way NE & NE 45th St,47.662020,-122.313196,4bf58dd8d48988d1fe931735,Bus Station
8,4518 Arcade,5a5f9be231ac6c6dbfc68220,Just Burgers,16,4510 University Way NE,47.661861,-122.312910,4bf58dd8d48988d16c941735,Burger Joint
9,4518 Arcade,5d3cad7742d74c0008eb3ea9,DL B.B.Q.,13,4520 the ave,47.661947,-122.313203,4bf58dd8d48988d1df931735,BBQ Joint


In [26]:
count = -1
distance_drop_list = list()
for dist in surrounding_venues['Venue_Distance']:
    count = count + 1
    if dist == 0:
        distance_drop_list.append(count)
print(len(distance_drop_list))

14


In [27]:
surrounding_venues = surrounding_venues.drop(index = distance_drop_list)
print(len(surrounding_venues))
surrounding_venues.head()

286


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,4518 Arcade,4ef5323b77c8e9126478ff6f,U:DON Fresh Japanese Noodle Station,7,4515 University Way NE,47.661795,-122.313194,55a59bace4b013909087cb2a,Udon Restaurant
1,4518 Arcade,4ae4f7d0f964a520c59f21e3,Urban Outfitters,45,4518 University Way Northeast,47.661430,-122.313030,4bf58dd8d48988d103951735,Clothing Store
2,4518 Arcade,59c14ab61ffed75cd77ec327,Chi Mac,25,4525 University Way NE,47.662044,-122.313256,4bf58dd8d48988d113941735,Korean Restaurant
3,4518 Arcade,5abb2319b040560b822d9633,Taste of Xi'an,21,4523 University Way NE,47.661957,-122.313337,4bf58dd8d48988d145941735,Chinese Restaurant
4,4518 Arcade,5f0e45adc5d2fb4660d1b12e,Don't Yell At Me 不要對我尖叫,21,4511 University Way NE,47.661674,-122.313270,52e81612bcbc57f1066b7a0c,Bubble Tea Shop


### Sorting by Arcade and Venue Distance

In [28]:
surrounding_venues = surrounding_venues.sort_values(['Arcade', 'Venue_Distance'],ascending = True)
surrounding_venues.head(10)

,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
5,4518 Arcade,5e23b504daee0b0008f60baa,Sugar+Spoon Cookie Dough,5,No Address,47.661790,-122.313146,4bf58dd8d48988d1c9941735,Ice Cream Shop
6,4518 Arcade,4b58fe5ff964a5205e7728e3,Deep Roots Tattoo & Body Piercing,5,4517 University Way NE,47.661806,-122.313184,4bf58dd8d48988d1de931735,Tattoo Parlor
0,4518 Arcade,4ef5323b77c8e9126478ff6f,U:DON Fresh Japanese Noodle Station,7,4515 University Way NE,47.661795,-122.313194,55a59bace4b013909087cb2a,Udon Restaurant
9,4518 Arcade,5d3cad7742d74c0008eb3ea9,DL B.B.Q.,13,4520 the ave,47.661947,-122.313203,4bf58dd8d48988d1df931735,BBQ Joint
8,4518 Arcade,5a5f9be231ac6c6dbfc68220,Just Burgers,16,4510 University Way NE,47.661861,-122.312910,4bf58dd8d48988d16c941735,Burger Joint
3,4518 Arcade,5abb2319b040560b822d9633,Taste of Xi'an,21,4523 University Way NE,47.661957,-122.313337,4bf58dd8d48988d145941735,Chinese Restaurant
4,4518 Arcade,5f0e45adc5d2fb4660d1b12e,Don't Yell At Me 不要對我尖叫,21,4511 University Way NE,47.661674,-122.313270,52e81612bcbc57f1066b7a0c,Bubble Tea Shop
7,4518 Arcade,4c37a5ae93db0f47db2f2092,Metro Bus Stop #9584,21,University Way NE & NE 45th St,47.662020,-122.313196,4bf58dd8d48988d1fe931735,Bus Station
14,4518 Arcade,435c2400f964a5200a291fe3,Yunnie Bubble Tea,22,4511 University Way NE,47.661647,-122.313221,52e81612bcbc57f1066b7a0c,Bubble Tea Shop
2,4518 Arcade,59c14ab61ffed75cd77ec327,Chi Mac,25,4525 University Way NE,47.662044,-122.313256,4bf58dd8d48988d113941735,Korean Restaurant


In [29]:
surrounding_venues.to_csv('surrounding_venues_sorted.csv')

### Ensuring Unique Venues

In [30]:
surrounding_sorted = pd.read_csv('surrounding_venues_sorted.csv')
surrounding_sorted = surrounding_sorted.drop('Unnamed: 0', axis= 1)
print(len(surrounding_sorted))
surrounding_sorted.head(20)

286


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,4518 Arcade,5e23b504daee0b0008f60baa,Sugar+Spoon Cookie Dough,5,No Address,47.661790,-122.313146,4bf58dd8d48988d1c9941735,Ice Cream Shop
1,4518 Arcade,4b58fe5ff964a5205e7728e3,Deep Roots Tattoo & Body Piercing,5,4517 University Way NE,47.661806,-122.313184,4bf58dd8d48988d1de931735,Tattoo Parlor
2,4518 Arcade,4ef5323b77c8e9126478ff6f,U:DON Fresh Japanese Noodle Station,7,4515 University Way NE,47.661795,-122.313194,55a59bace4b013909087cb2a,Udon Restaurant
3,4518 Arcade,5d3cad7742d74c0008eb3ea9,DL B.B.Q.,13,4520 the ave,47.661947,-122.313203,4bf58dd8d48988d1df931735,BBQ Joint
4,4518 Arcade,5a5f9be231ac6c6dbfc68220,Just Burgers,16,4510 University Way NE,47.661861,-122.312910,4bf58dd8d48988d16c941735,Burger Joint
5,4518 Arcade,5abb2319b040560b822d9633,Taste of Xi'an,21,4523 University Way NE,47.661957,-122.313337,4bf58dd8d48988d145941735,Chinese Restaurant
6,4518 Arcade,5f0e45adc5d2fb4660d1b12e,Don't Yell At Me 不要對我尖叫,21,4511 University Way NE,47.661674,-122.313270,52e81612bcbc57f1066b7a0c,Bubble Tea Shop
7,4518 Arcade,4c37a5ae93db0f47db2f2092,Metro Bus Stop #9584,21,University Way NE & NE 45th St,47.662020,-122.313196,4bf58dd8d48988d1fe931735,Bus Station
8,4518 Arcade,435c2400f964a5200a291fe3,Yunnie Bubble Tea,22,4511 University Way NE,47.661647,-122.313221,52e81612bcbc57f1066b7a0c,Bubble Tea Shop
9,4518 Arcade,59c14ab61ffed75cd77ec327,Chi Mac,25,4525 University Way NE,47.662044,-122.313256,4bf58dd8d48988d113941735,Korean Restaurant


### Pulling the Ratings

In [33]:
count = -1
rating_list = list()
for ident in surrounding_sorted['Venue_ID']:
    count = count +1
    venue_id = surrounding_sorted['Venue_ID'][count]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        rating = result['response']['venue']['rating']
    except:
        rating = 'Not Rated'
    rating_list.append(rating)
rating_list

['Not Rated',
 7.7,
 7.7,
 'Not Rated',
 'Not Rated',
 7.8,
 'Not Rated',
 'Not Rated',
 6.3,
 7.7,
 7.8,
 'Not Rated',
 5.5,
 'Not Rated',
 7.9,
 7.0,
 'Not Rated',
 5.4,
 'Not Rated',
 8.2,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 8.4,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 6.2,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 9.1,
 'Not Rated',
 7.0,
 'Not Rated',
 7.9,
 8.5,
 7.0,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 7.7,
 6.7,
 'Not Rated',
 7.9,
 7.1,
 8.9,
 8.3,
 'Not Rated',
 8.4,
 9.3,
 'Not Rated',
 'Not Rated',
 5.4,
 7.6,
 8.4,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 7.2,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 5.8,
 7.8,
 8.6,
 'Not Rated',
 'Not Rated',
 8.6,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 7.8,
 6.9,
 6.4,
 8.2,
 7.5,
 'Not Rated',
 7.2,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 7.6,
 'Not Rated'

In [34]:
surrounding_final = surrounding_sorted.assign(Venue_Rating= rating_list)
surrounding_final.head()

,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,4518 Arcade,5e23b504daee0b0008f60baa,Sugar+Spoon Cookie Dough,5,No Address,47.661790,-122.313146,4bf58dd8d48988d1c9941735,Ice Cream Shop,Not Rated
1,4518 Arcade,4b58fe5ff964a5205e7728e3,Deep Roots Tattoo & Body Piercing,5,4517 University Way NE,47.661806,-122.313184,4bf58dd8d48988d1de931735,Tattoo Parlor,7.7
2,4518 Arcade,4ef5323b77c8e9126478ff6f,U:DON Fresh Japanese Noodle Station,7,4515 University Way NE,47.661795,-122.313194,55a59bace4b013909087cb2a,Udon Restaurant,7.7
3,4518 Arcade,5d3cad7742d74c0008eb3ea9,DL B.B.Q.,13,4520 the ave,47.661947,-122.313203,4bf58dd8d48988d1df931735,BBQ Joint,Not Rated
4,4518 Arcade,5a5f9be231ac6c6dbfc68220,Just Burgers,16,4510 University Way NE,47.661861,-122.312910,4bf58dd8d48988d16c941735,Burger Joint,Not Rated


In [35]:
counts = dict()
for category in surrounding_final['Category_Name']:
    counts[category]= counts.get(category, 0) + 1

In [36]:
counts

{'Ice Cream Shop': 2,
 'Tattoo Parlor': 1,
 'Udon Restaurant': 1,
 'BBQ Joint': 3,
 'Burger Joint': 1,
 'Chinese Restaurant': 3,
 'Bubble Tea Shop': 3,
 'Bus Station': 3,
 'Korean Restaurant': 3,
 'Cosmetics Shop': 2,
 'Szechuan Restaurant': 1,
 'Tea Room': 1,
 'Sandwich Place': 4,
 'Clothing Store': 3,
 'Thai Restaurant': 1,
 'Tech Startup': 5,
 'Music Store': 1,
 'Bar': 4,
 'Food Truck': 1,
 'Smoke Shop': 1,
 'Office': 8,
 'Bus Line': 3,
 'Event Space': 3,
 'Art Gallery': 5,
 'Bus Stop': 3,
 'Factory': 1,
 'Trail': 3,
 'Beer Garden': 1,
 'Park': 7,
 'Marijuana Dispensary': 3,
 'Post Office': 3,
 'School': 1,
 'Mexican Restaurant': 3,
 'Shoe Repair': 1,
 'Coffee Shop': 8,
 'Salon / Barbershop': 5,
 'Seafood Restaurant': 4,
 'Gift Shop': 3,
 'Lounge': 1,
 'Bookstore': 5,
 'Pharmacy': 1,
 'Cocktail Bar': 2,
 'Conference Room': 2,
 'Arts & Crafts Store': 1,
 'Dive Bar': 2,
 'General Entertainment': 2,
 'Building': 6,
 'Mental Health Office': 1,
 'Rock Club': 1,
 'Mediterranean Restaurant

In [37]:
surrounding_final.to_csv('surrounding_final.csv')

### prepping for clustering

In [24]:
surrounding_split = pd.read_csv('surrounding_final.csv')
surrounding_split = surrounding_split.drop('Unnamed: 0', axis = 1)
print(len(surrounding_split))
surrounding_split.head()

286


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,4518 Arcade,5e23b504daee0b0008f60baa,Sugar+Spoon Cookie Dough,5,No Address,47.661790,-122.313146,4bf58dd8d48988d1c9941735,Ice Cream Shop,Not Rated
1,4518 Arcade,4b58fe5ff964a5205e7728e3,Deep Roots Tattoo & Body Piercing,5,4517 University Way NE,47.661806,-122.313184,4bf58dd8d48988d1de931735,Tattoo Parlor,7.7
2,4518 Arcade,4ef5323b77c8e9126478ff6f,U:DON Fresh Japanese Noodle Station,7,4515 University Way NE,47.661795,-122.313194,55a59bace4b013909087cb2a,Udon Restaurant,7.7
3,4518 Arcade,5d3cad7742d74c0008eb3ea9,DL B.B.Q.,13,4520 the ave,47.661947,-122.313203,4bf58dd8d48988d1df931735,BBQ Joint,Not Rated
4,4518 Arcade,5a5f9be231ac6c6dbfc68220,Just Burgers,16,4510 University Way NE,47.661861,-122.312910,4bf58dd8d48988d16c941735,Burger Joint,Not Rated


In [25]:
venue_clustering_df = pd.DataFrame(surrounding_split[['Venue_Distance', 'Category_ID', 'Category_Name', 'Venue_Rating']])
venue_clustering_df.head()

,Venue_Distance,Category_ID,Category_Name,Venue_Rating
0,5,4bf58dd8d48988d1c9941735,Ice Cream Shop,Not Rated
1,5,4bf58dd8d48988d1de931735,Tattoo Parlor,7.7
2,7,55a59bace4b013909087cb2a,Udon Restaurant,7.7
3,13,4bf58dd8d48988d1df931735,BBQ Joint,Not Rated
4,16,4bf58dd8d48988d16c941735,Burger Joint,Not Rated


In [26]:
venue_clustering_df.to_csv('seattle_clustering_category.csv')

In [21]:
arcade_venue_df = pd.DataFrame(surrounding_split[['Arcade', 'Venue_ID', 'Venue_Name', 'Venue_Address']])
arcade_venue_df.insert(0, 'City', 'Null')
arcade_venue_df = arcade_venue_df.assign(City = 'Seattle, WA')
arcade_venue_df.head()

,City,Arcade,Venue_ID,Venue_Name,Venue_Address
0,"Seattle, WA",4518 Arcade,5e23b504daee0b0008f60baa,Sugar+Spoon Cookie Dough,No Address
1,"Seattle, WA",4518 Arcade,4b58fe5ff964a5205e7728e3,Deep Roots Tattoo & Body Piercing,4517 University Way NE
2,"Seattle, WA",4518 Arcade,4ef5323b77c8e9126478ff6f,U:DON Fresh Japanese Noodle Station,4515 University Way NE
3,"Seattle, WA",4518 Arcade,5d3cad7742d74c0008eb3ea9,DL B.B.Q.,4520 the ave
4,"Seattle, WA",4518 Arcade,5a5f9be231ac6c6dbfc68220,Just Burgers,4510 University Way NE


In [22]:
arcade_venue_df.to_csv('seattle_no_cluster_arcade_data.csv')